In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
from gensim.models import KeyedVectors

In [4]:
w2v=KeyedVectors.load("/content/gdrive/MyDrive/FYP/IMDb/Word2Vec/w2c")

In [5]:
import pandas as pd
import numpy as np
df=pd.read_csv("/content/gdrive/MyDrive/FYP/IMDb/IMDB-preprocessed.csv")
y=df["sentiment"]


In [6]:
maxi=-1
for i,rev in enumerate(df['new_text']):
  tokens=rev.split()
  if(len(tokens)>maxi):
    maxi=len(tokens)
print(maxi)

2511


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import tensorflow as tf
maxlen=66
max_words=len(w2v.wv.vocab.keys())
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_words)
def padded_sequence(X):
  tokenizer.fit_on_texts(X)
  sequences = tokenizer.texts_to_sequences(X)
  data = pad_sequences(sequences, maxlen=maxlen)
  return data
X=padded_sequence(df["new_text"])
X

array([[   19,  7608,  7149, ...,   123,  4013,   486],
       [  203,     2,    27, ...,  1900,    75,   216],
       [   10,   132,     3, ...,    67,    16,   333],
       ...,
       [ 3880,    70,   148, ..., 22558,     2,  5887],
       [   15,     1,   341, ...,    49,   695,    44],
       [    9,   154,    18, ...,   759,    11,    18]], dtype=int32)

In [9]:
import numpy as np
embed_matrix=np.zeros(shape=(max_words,300))
d={}
for word,i in tokenizer.word_index.items():
  d[i]=word

In [10]:
for i in X:
  for j in range(len(i)):
    if i[j]==0:
      pass
    else:
      p=d[i[j]]
      if p in w2v:
        i[j]=sum(w2v[p])
      else:
        i[j]=0
  print(i)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


Streaming output truncated to the last 5000 lines.
[ 11 -11 -26 -50  21   1   8 -42   9 -15 -20 -17 -69  48  14   7 -35   8
 -50   2   1  -3   7  12   8 -10 -10  -1  14  11  37   4  -6   4  19  32
 -17 -22  16 -13   4  39 -10 -24   4 -25  -1   0  13 -42 -14 -38 -10 -10
 -30   6   8   0   4 -14 -21 -86 -50 -12  10  23]
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
  -6 -30 -25  10   7   6  26 -31 -44 -53 -16  28  -1 -16 -16   7 -43 -25
  -9  11  16   8  -8  40  -9  -6   8  47   0  41 -17  20 -17  10  -1  16
  13   7   9 -22   8 -30 -25 -10 -10   6 -10 -10]
[-24  -6  -2  -5  28   4   9  19  16   3 -44 -35  13 -36  16  10   9 -15
  11   4  -6   3  16  15 -39 -17   8   3 -19 -15  10 -15 -30 -21  13   7
   7  -6   3  25  21  16 -16  -9  -6   3   4  37   5  16  -6   4 -18 -24
 -11  28 -12   8  -1  10 -30  23  39  16  10 -42]
[  0   0   0   0   0   4  25  15 -38 -24  -2   0  20 -13   8  -9   3  19
  -7   4  27  10 -30  22   0  23   8   0   1 -15 -24  17   3  16   5 

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [12]:
#Metric calculation function
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
def display_metrics(y_test,y_pred):
    print(accuracy_score(y_test, y_pred))
    print(f1_score(y_test, y_pred, average="macro"))
    print(precision_score(y_test, y_pred, average="macro"))
    print(recall_score(y_test, y_pred, average="macro"))

In [13]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(max_iter=10000).fit(X_train, y_train)
y_pred=clf.predict(X_test)

In [14]:
display_metrics(y_test,y_pred)

0.55272
0.5527182108728435
0.5528793162649517
0.5528721738606521


In [15]:
from sklearn import svm 
clf = svm.SVC(kernel='linear',verbose=1,max_iter=10000)
clf.fit(X_train, y_train)

[LibSVM]

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,


SVC(kernel='linear', max_iter=10000, verbose=1)

In [16]:
y_pred=clf.predict(X_test)
display_metrics(y_test,y_pred)

0.5312
0.5229889997681695
0.5312745206881693
0.5293649530234548


In [17]:
from sklearn import tree
DTclf = tree.DecisionTreeClassifier()
DTclf.fit(X_train, y_train)

DecisionTreeClassifier()

In [18]:
y_pred=DTclf.predict(X_test)
display_metrics(y_test,y_pred)

0.52008
0.5200693079120695
0.5201210690548692
0.5201249295492012
